In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!wget https://repo.anaconda.com/archive/Anaconda3-2018.12-Linux-x86_64.sh
!chmod +x Anaconda3-2018.12-Linux-x86_64.sh
!bash ./Anaconda3-2018.12-Linux-x86_64.sh -b -f -p /usr/local

In [0]:
!conda create -n tensorflow1 pip python=3.5
!activate tensorflow1
!pip install --ignore-installed --upgrade tensorflow-gpu
!conda install -c anaconda protobuf
!pip install opencv-python
!cd /content/
!cp gdrive/'My Drive'/tensorflow1.zip /content
!unzip tensorflow1.zip
%set PYTHONPATH=\content\tensorflow1\models;\content\tensorflow1\models\research;\content\tensorflow1\models\research\slim
!rm tensorflow1.zip
!cd tensorflow1/models/research
!protoc --python_out=. ./object_detection/protos/anchor_generator.proto ./object_detection/protos/argmax_matcher.proto ./object_detection/protos/bipartite_matcher.proto ./object_detection/protos/box_coder.proto ./object_detection/protos/box_predictor.proto ./object_detection/protos/eval.proto ./object_detection/protos/faster_rcnn.proto ./object_detection/protos/faster_rcnn_box_coder.proto ./object_detection/protos/grid_anchor_generator.proto ./object_detection/protos/hyperparams.proto ./object_detection/protos/image_resizer.proto ./object_detection/protos/input_reader.proto ./object_detection/protos/losses.proto ./object_detection/protos/matcher.proto ./object_detection/protos/mean_stddev_box_coder.proto ./object_detection/protos/model.proto ./object_detection/protos/optimizer.proto ./object_detection/protos/pipeline.proto ./object_detection/protos/post_processing.proto ./object_detection/protos/preprocessor.proto ./object_detection/protos/region_similarity_calculator.proto ./object_detection/protos/square_box_coder.proto ./object_detection/protos/ssd.proto ./object_detection/protos/ssd_anchor_generator.proto ./object_detection/protos/string_int_label_map.proto ./object_detection/protos/train.proto ./object_detection/protos/keypoint_box_coder.proto ./object_detection/protos/multiscale_anchor_generator.proto ./object_detection/protos/graph_rewriter.proto
!python setup.py build
!python setup.py install

In [0]:
cd tensorflow1/models/research/object_detection

In [0]:
ls

In [0]:
!python sizeChecker.py --move

In [0]:
!pip uninstall numpy
!pip uninstall numpy

In [0]:
!pip install numpy

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
model_dir = 'training/'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!ls {model_dir}

In [0]:
!cp -a /content/gdrive/'My Drive'/training/ ./

In [0]:
ls

In [0]:
!python ./export_inference_graph.py --input_type image_tensor --pipeline_config_path ./training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix ./training/model.ckpt-60174 --output_directory inference_graph

In [0]:
cd ..

In [0]:
!python setup.py build
!python setup.py install

In [0]:
cd ..

In [0]:
ls

In [0]:
!python setup.py build
!python setup.py install

In [0]:
!python ./export_inference_graph.py --input_type image_tensor --pipeline_config_path ./training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix ./training/model.ckpt-60174 --output_directory inference_graph

In [0]:
cd object_detection

In [0]:
!python ./export_inference_graph.py --input_type image_tensor --pipeline_config_path ./training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix ./training/model.ckpt-57102 --output_directory inference_graph

In [0]:
%set_env PYTHONPATH=\content\tensorflow1\models;\content\tensorflow1\models\research;\content\tensorflow1\models\research\slim

In [0]:
!python ./export_inference_graph.py --input_type image_tensor --pipeline_config_path ./training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix ./training/model.ckpt-57102 --output_directory inference_graph

In [0]:
ls

In [0]:
!cp -r ../slim/nets /usr/local/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/
!cp -r ../slim/deployment /usr/local/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/
!python ./export_inference_graph.py --input_type image_tensor --pipeline_config_path ./training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix ./training/model.ckpt-60174 --output_directory inference_graph

In [0]:
import os
import glob

pb_fname = '/content/tensorflow1/models/research/object_detection/inference_graph/frozen_inference_graph.pb'
label_map_pbtxt_fname = '/content/tensorflow1/models/research/object_detection/training/labelmap.pbtxt'
repo_dir_path = '/content/tensorflow1/models/research/object_detection/images/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "testing")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [0]:
%cd /content/tensorflow1/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

        
num_classes = 20
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)